In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(1, os.path.dirname(os.getcwd()))


import numpy as np
from utils.network_graph import DirectedGraph, graph_dict_3x3, graph_dict_3x3_2,\
                                graph_dict_4_nodes, create_nxn_graph
from models.O_flows_linear import OFlowsLinear

In [2]:
# for n = 6 it takes some time (about 10-20 minutes on 2022 Macbook M1 pro 16 gb ram)
# 7 and 8 - too much time

n = 6
graph_nxn_bi = DirectedGraph(create_nxn_graph(n))

In [4]:
n_t = 65 # number of intervals

links = graph_nxn_bi.get_links()
o_nodes = graph_nxn_bi.get_o_nodes()
num_o_nodes = len(o_nodes)
num_links = len(links)

P_o = graph_nxn_bi.generate_o_assignment_matrix()
o_flows_mat = np.zeros((num_o_nodes, n_t))
link_flows_mat = np.zeros((num_links, n_t))
for i in range(n_t):
    o_flows = graph_nxn_bi.generate_o_flows()[0]
    link_flows = P_o @ o_flows

    o_flows_mat[:, i] = o_flows
    link_flows_mat[:, i] = link_flows

In [ ]:
model = OFlowsLinear(graph_nxn_bi)

In [ ]:
opt_P, opt_x = model.fit(n_t, link_flows_mat, num_inits=1, threshold=1e-5, verbose=True)

is anything going on?
is anything going on?
is anything going on?
NMSE, iteration 2:  437.3603454163579
is anything going on?
NMSE, iteration 3:  116.5364179258885
is anything going on?
NMSE, iteration 4:  51.01813922633833
is anything going on?
NMSE, iteration 5:  30.952675530185175
is anything going on?
NMSE, iteration 6:  21.889210907142886


In [ ]:
# some statistics

mock_x = np.random.rand(*o_flows_mat.shape) * 1000
mock_P = DirectedGraph(create_nxn_graph(n)).generate_o_assignment_matrix()

print("NMSE opt_P @ opt_x   : ", np.round((np.linalg.norm(opt_P @ opt_x - link_flows_mat) / np.linalg.norm(link_flows_mat)) ** 2, 6))
print("NMSE mock_P @ opt_x  : ", np.round((np.linalg.norm(mock_P @ opt_x - link_flows_mat) / np.linalg.norm(link_flows_mat)) ** 2, 6))
print("NMSE opt_P @ mock_x  : ", np.round((np.linalg.norm(opt_P @ mock_x - link_flows_mat) / np.linalg.norm(link_flows_mat)) ** 2, 6))
print("NMSE mock_P @ mock_x : ", np.round((np.linalg.norm(mock_P @ mock_x - link_flows_mat) / np.linalg.norm(link_flows_mat)) ** 2, 6))

NMSE opt_P @ opt_x   :  9e-06
NMSE mock_P @ opt_x  :  0.017382
NMSE opt_P @ mock_x  :  0.121333
NMSE mock_P @ mock_x :  0.132177


In [ ]:
# basic visual test
# compare ground truth and predicted od flows for the first interval

o_flows_mat[:, 0], opt_x[:, 0]